# Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

## Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

## Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

### urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [2]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [3]:
soup.prettify()[0:1000]

u'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml">\n <head>\n  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">\n  </script>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>\n  <meta content="#CC0000" name="theme-color"/>\n  <script type="text/javascript">\n   if(true &amp;&amp; navigator &amp;&amp; navigator.userAgent.toLowerCase().indexOf("teamstream") &gt;= 0) {\n        window.location = \'http://a.m.espn.go.com/mobilecache/general/apps/sc\';\n    }\n  </script>\n  <title>\n   2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN\n  </title>\n  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>\n  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>\n  <meta content="noodp" name="googl

## Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

### find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [4]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [5]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

### Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [6]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [7]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [8]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


### Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [9]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [10]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

[{u'FGM-FGA': u'11.7-20.5', u'MPG': u'32.3', u'FTM-FTA': u'6.2-6.7', u'FG%': u'.569', u'GP': u'6', u'PLAYER': u'Stephen Curry', u'FT%': u'.925', u'TEAM': u'GS', u'3PM-3PA': u'6.0-11.7', 'POSITION': u'PG', u'PTS': u'35.5', u'3P%': u'.514'}, {u'FGM-FGA': u'10.2-21.0', u'MPG': u'38.2', u'FTM-FTA': u'6.7-7.5', u'FG%': u'.484', u'GP': u'6', u'PLAYER': u'Kevin Durant', u'FT%': u'.889', u'TEAM': u'OKC', u'3PM-3PA': u'2.8-6.7', 'POSITION': u'SF', u'PTS': u'29.8', u'3P%': u'.425'}, {u'FGM-FGA': u'11.2-19.0', u'MPG': u'34.6', u'FTM-FTA': u'5.8-7.6', u'FG%': u'.589', u'GP': u'5', u'PLAYER': u'Blake Griffin', u'FT%': u'.763', u'TEAM': u'LAC', u'3PM-3PA': u'0.0-0.4', 'POSITION': u'PF', u'PTS': u'28.2', u'3P%': u'.000'}, {u'FGM-FGA': u'10.0-21.3', u'MPG': u'36.7', u'FTM-FTA': u'3.5-4.0', u'FG%': u'.469', u'GP': u'6', u'PLAYER': u'Damian Lillard', u'FT%': u'.875', u'TEAM': u'POR', u'3PM-3PA': u'3.8-9.0', 'POSITION': u'PG', u'PTS': u'27.3', u'3P%': u'.426'}, {u'FGM-FGA': u'10.2-21.2', u'MPG': u'36.0',

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [11]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.514,6.0-11.7,.569,11.7-20.5,.925,6.2-6.7,6,32.3,Stephen Curry,PG,35.5,GS
1,.425,2.8-6.7,.484,10.2-21.0,.889,6.7-7.5,6,38.2,Kevin Durant,SF,29.8,OKC
2,.000,0.0-0.4,.589,11.2-19.0,.763,5.8-7.6,5,34.6,Blake Griffin,PF,28.2,LAC
3,.426,3.8-9.0,.469,10.0-21.3,.875,3.5-4.0,6,36.7,Damian Lillard,PG,27.3,POR
4,.344,1.8-5.3,.480,10.2-21.2,.857,5.0-5.8,6,36.0,Russell Westbrook,PG,27.2,OKC


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [16]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()


,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Stephen Curry,PG,GS,6,32.3,35.5,11.7-20.5,.569,6.0-11.7,.514,6.2-6.7,.925
1,Kevin Durant,SF,OKC,6,38.2,29.8,10.2-21.0,.484,2.8-6.7,.425,6.7-7.5,.889
2,Blake Griffin,PF,LAC,5,34.6,28.2,11.2-19.0,.589,0.0-0.4,.000,5.8-7.6,.763
3,Damian Lillard,PG,POR,6,36.7,27.3,10.0-21.3,.469,3.8-9.0,.426,3.5-4.0,.875
4,Russell Westbrook,PG,OKC,6,36.0,27.2,10.2-21.2,.480,1.8-5.3,.344,5.0-5.8,.857


## Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [2]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np

def get_cols(header_cols):
    ''' Retrieve the names of the header columns given a table element
    (skip the ranks, split the positions) '''
    
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

def get_data(table_rows):
    ''' Retrieve the actual table data given a table element
    (skip the ranks, split the positions)'''
    
    players_stats_array = []
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
    return players_stats_array

def create_url_list(base_url, counts):
    urls = []
    for count in counts:
        urls.append(base_url + str(count))
    return urls

def create_data_frame(arr, cols):
    np_array = np.array(arr)
    return pd.DataFrame(np_array, columns=cols)

def make_soup(url):
    page = urllib2.urlopen(url).read()
    return BeautifulSoup(page)

def get_table(soup):
    table_div = soup.find(id='my-players-table')
    return table_div.find("table")

def get_headers(table):
    table_head = table.find(attrs={"class":'colhead'})
    return table_head.findAll('td')

def get_espn_data():
    page_counts = range(1, 402, 40)

    ''' Create the scores data frame '''
    scoring_base_url = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/'
    scoring_urls = create_url_list(scoring_base_url, page_counts)
    scoring_array = []
    for url in scoring_urls:
        soup = make_soup(url)
        table = get_table(soup)
        table_rows = table.findAll('tr')
        if 'count/1' in url:
            header_cols = get_headers(table)
            cols = get_cols(header_cols)
        scoring_array += get_data(table_rows)

    scoring_df = create_data_frame(scoring_array, cols)

    
    ''' Create the assists data frame '''
    assists_base_url = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/'
    assists_urls = create_url_list(assists_base_url, page_counts)
    assists_array = []
    for url in assists_urls:
        soup = make_soup(url)
        table = get_table(soup)
        table_rows = table.findAll('tr')
        if 'count/1' in url:
            header_cols = get_headers(table)
            cols = get_cols(header_cols)
        assists_array += get_data(table_rows)

    assists_df = create_data_frame(assists_array, cols)

    ''' Remove duplicate columns '''
    cols_to_use = assists_df.columns.difference(scoring_df.columns)
    
    ''' Return the merged scoring and assists data frames '''
    return pd.merge(scoring_df, assists_df[cols_to_use], right_index=True, left_index=True, how="outer")


print get_espn_data()

                       PLAYER POSITION  TEAM GP   MPG   PTS    FGM-FGA   FG%  \
0               Stephen Curry       PG    GS  8  33.8  32.4  10.6-19.9  .535   
1                James Harden       SG   HOU  7  39.4  29.7   8.1-21.6  .377   
2               Blake Griffin       PF   LAC  7  35.6  28.6  11.3-19.4  .581   
3                Kevin Durant       SF   OKC  8  34.8  28.1   9.6-19.1  .503   
4              Damian Lillard       PG   POR  8  36.6  27.5   9.6-20.6  .467   
5           Russell Westbrook       PG   OKC  8  34.8  25.8   9.6-20.5  .470   
6                LeBron James       SF   CLE  8  34.8  25.3  10.0-20.0  .500   
7               Anthony Davis       PF    NO  7  35.4  24.0   8.3-17.4  .475   
8                 Paul George       SF   IND  8  35.9  23.5   7.5-17.9  .420   
9              Reggie Jackson       PG   DET  7  32.7  22.7   8.3-18.7  .443   
10               Bradley Beal       SG   WSH  6  34.7  22.7   8.7-17.8  .486   
11              DeMar DeRozan       SG  